In [1]:
import pandas as pd
import cPickle as pkl
import haversine
import re

In [2]:
geo_data = pkl.load(open('../data/geotag_imglink_wikiurl.pkl','rb'))


It says in the read me that...

          Variable          Columns      Type
          --------          -------      ----

          ID                 1-11        Integer
          YEAR              12-15        Integer
          ELEMENT           16-19        Character
          VALUE1            20-24        Integer
          DMFLAG1           25-25        Character
          QCFLAG1           26-26        Character
          DSFLAG1           27-27        Character
            .                 .             .
            .                 .             .
            .                 .             .
          VALUE12          108-112       Integer
          DMFLAG12         113-113       Character
          QCFLAG12         114-114       Character
          DSFLAG12         115-115       Character

Lets first get the ID, YEAR, Element


In [3]:
widths = [19,150]
globaldata = pd.read_fwf('../data/weather/ghcnm.tavg.v3.3.0.20150624.qca.dat',header = None, widths=widths)

In [4]:
globaldata.head()

,0,1
0,101603550001878TAVG,956 1 1016 1 1176 1 1676 1 2046 1 2306 1...
1,101603550001879TAVG,1246 1 1236 1 1286 1 1616 1 1626 1 2336 ...
2,101603550001880TAVG,1026 1 1176 1 1306 1 1586 1 1776 1 2136 ...
3,101603550001931TAVG,-9999 1036 1-9999 -9999 1916 1 2456 ...
4,101603550001932TAVG,1076 1 1046 1-9999 1486 1 1906 1-9999 ...


In [5]:
# Lets parse the first part of the data to stationid, year, and measurement type.
globaldata['station_id'] = globaldata[0].apply(lambda x: int(x[:11]))
globaldata['year'] = globaldata[0].apply(lambda x: int(x[11:15]))
globaldata['type'] = globaldata[0].apply(lambda x: x[15:19])
globaldata = globaldata.drop(0,axis = 1)

In [6]:
# Look at what we have now.
globaldata.head()

,1,station_id,year,type
0,956 1 1016 1 1176 1 1676 1 2046 1 2306 1...,10160355000,1878,TAVG
1,1246 1 1236 1 1286 1 1616 1 1626 1 2336 ...,10160355000,1879,TAVG
2,1026 1 1176 1 1306 1 1586 1 1776 1 2136 ...,10160355000,1880,TAVG
3,-9999 1036 1-9999 -9999 1916 1 2456 ...,10160355000,1931,TAVG
4,1076 1 1046 1-9999 1486 1 1906 1-9999 ...,10160355000,1932,TAVG


In [7]:
#Seems like it has a lot of old data....lets see which year's the most recent data
globaldata.groupby('station_id')['year'].max().head()

station_id
10160355000    2015
10160360000    2015
10160390000    2015
10160395001    1961
10160400001    1960
Name: year, dtype: int64

In [8]:
#since 2015 isn't complete yet, lets use 5 years data to 2014. We could use up to June 2015 but lets leave out 2015 to make it easier.
condition1 = globaldata['year'] >= 1990
condition2 = globaldata['year']<2015

In [9]:
globaldata = globaldata[condition1 & condition2]

In [10]:
globaldata.tail(10)

,1,station_id,year,type
462505,82 M -18 M -158 M -198 M -508 M -808 M-...,70188968000,2007,TAVG
462506,112 M 102 M -108 M -308 M -528 M-1058 M...,70188968000,2008,TAVG
462507,152 M 72 M 92 M -158 M -278 M -668 M...,70188968000,2009,TAVG
462508,42 M 22 M -58 M -348 M -598 M -388 M ...,70188968000,2010,TAVG
462509,132 M 212 M 162 M -168 M -318 M -308 M...,70188968000,2011,TAVG
462510,110 M 100 M -70 M -330 M -610 M -780 M...,70188968000,2012,TAVG
462511,-10 M 30 M -100 M -60 M -60 M -380 M...,70188968000,2013,TAVG
462512,10 C 60 C -30 C -200 C -400 C -680 C ...,70188968000,2014,TAVG
462614,320 G 120 G-9999 -9999 -9999 -9999 ...,80099903001,1990,TAVG
462809,-9999 XG-9999 XG-9999 -9999 -9999 -9999 ...,80099910001,1990,TAVG


In [11]:
globaldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77122 entries, 38 to 462809
Data columns (total 4 columns):
1             77122 non-null object
station_id    77122 non-null int64
year          77122 non-null int64
type          77122 non-null object
dtypes: int64(2), object(2)
memory usage: 2.9+ MB


We have 16443 rows of data...so lets parse the temperature part.
README says that temperature is followed by 3 flags and it also says that -9999 means missing data. Lets first collect all the numbers. There should be 12 entries for each row.


In [12]:
def parse_temp(string):
    no_alpha = re.sub('[a-zA-Z]', '',string)
    no_single_number = re.sub('\s+\d\s+',' ',no_alpha)
    no_inbetween_hypthens = re.sub('(\d+-)','',no_single_number)
    
    return no_inbetween_hypthens.split()

In [13]:
parse_temp('012 -313 3 3-133 D S 1029')

['012', '-313', '133', '1029']

In [14]:
global_temp = globaldata[1].apply(parse_temp)
global_temp.head()

38    [-9999, -9999, 1440, 1540, 1870, 2190, 2460, 2...
39    [-9999, 1120, 1510, 1400, 1590, 2110, 2430, 25...
40    [1090, 1170, 9999, 1500, 1780, 2090, 2310, 252...
41    [-9999, 1110, 1290, 1550, 1910, 2190, 2420, 25...
42    [-9999, 1320, 1430, 1450, 1950, 2160, 2530, 28...
Name: 1, dtype: object

In [15]:
temp_counts = globaldata[1].apply(lambda x: len(parse_temp(x)))
temp_counts.unique()

array([13, 12, 11, 10])

check out how many entries are not lengths 12

In [16]:
global_temp[temp_counts!= 12].count()

4454

Since there are 198 of these.... There may be temperatures with only single digits but those can't be distinguished with the temperatures so lets get rid of them

In [17]:
global_temp = global_temp[temp_counts==12]
globaldata = globaldata[temp_counts==12]

In [18]:
globaldata['avg_temps'] = global_temp

In [19]:
globaldata.head()

,1,station_id,year,type,avg_temps
42,-9999 1320 2 1430 2 1450 2 1950 2 2160 ...,10160355000,1994,TAVG,"[-9999, 1320, 1430, 1450, 1950, 2160, 2530, 28..."
43,1250 M 1410 M 1350 M-9999 1960 M 2160 ...,10160355000,1995,TAVG,"[1250, 1410, 1350, -9999, 1960, 2160, -9999, 2..."
44,-9999 -9999 1440 M 1680 M 1840 M 2150 ...,10160355000,1996,TAVG,"[-9999, -9999, 1440, 1680, 1840, 2150, 2450, 2..."
45,1520 M 1330 M 1360 M 1580 M 1980 M 2340 ...,10160355000,1997,TAVG,"[1520, 1330, 1360, 1580, 1980, 2340, 2430, 263..."
46,1300 M 1320 M 1460 M 1700 M 1870 M 2370 ...,10160355000,1998,TAVG,"[1300, 1320, 1460, 1700, 1870, 2370, 2500, 259..."


In [20]:
df_global_temp = global_temp.apply(lambda x: pd.Series(x))
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
df_global_temp.columns = months

In [21]:
df_global_temp.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
42,-9999,1320,1430,1450,1950,2160,2530,2820,2520,2110,-9999,1390
43,1250,1410,1350,-9999,1960,2160,-9999,2570,2320,-9999,1740,1570
44,-9999,-9999,1440,1680,1840,2150,2450,2570,2170,1840,1710,1520
45,1520,1330,1360,1580,1980,2340,2430,2630,-9999,2160,-9999,1460
46,1300,1320,1460,1700,1870,2370,2500,2590,2450,1920,1530,1250


Since these temperatures in hundredths of celcius degrees, divide them by 100 to get celcius degrees

In [22]:
def check_temp(temp):
    if temp == '-9999':
        output = None
    else:
        output = float(temp)/100
    return output

In [23]:
df_global_temp = df_global_temp.applymap(check_temp)

Lets merge the globaldata with the parsed temperatures.

In [24]:
station_temps = globaldata.merge(df_global_temp, left_index = True, right_index = True)
station_temps = station_temps.drop(1,axis=1)
station_temps = station_temps.drop('avg_temps',axis=1)

In [25]:
station_temps.head()

,station_id,year,type,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
42,10160355000,1994,TAVG,NaN,13.2,14.3,14.5,19.5,21.6,25.3,28.2,25.2,21.1,NaN,13.9
43,10160355000,1995,TAVG,12.5,14.1,13.5,NaN,19.6,21.6,NaN,25.7,23.2,NaN,17.4,15.7
44,10160355000,1996,TAVG,NaN,NaN,14.4,16.8,18.4,21.5,24.5,25.7,21.7,18.4,17.1,15.2
45,10160355000,1997,TAVG,15.2,13.3,13.6,15.8,19.8,23.4,24.3,26.3,NaN,21.6,NaN,14.6
46,10160355000,1998,TAVG,13.0,13.2,14.6,17.0,18.7,23.7,25.0,25.9,24.5,19.2,15.3,12.5


We finally have a parsed table of temperatures. So lets get the average monthly temperatures for the past 5 years for each weather station. Temperatures with -99.99 represent missing data so we won't incorporate these. If there are stations with only a year of data and that data is missing, let's not use that station. 

In [26]:
cond = station_temps.groupby('station_id')[months].count()==1

In [27]:
print station_temps.groupby('station_id')[months].mean().shape
print station_temps.groupby('station_id')[months].mean().dropna().shape

(4877, 12)
(4180, 12)


In [28]:
station_temps = station_temps.groupby('station_id')[months].mean().dropna()

In [29]:
print station_temps.head()
print station_temps.shape

                   Jan        Feb        Mar        Apr        May        Jun  \
station_id                                                                      
10160355000  12.541176  12.731250  14.538889  16.620000  19.500000  22.768421   
10160360000  11.159091  11.350000  13.127273  15.317391  18.513636  22.347826   
10160390000  11.020000  11.145238  13.392857  15.483333  18.742500  22.764286   
10160402000  11.385000  11.205000  13.605000  15.545000  18.475000  22.240909   
10160403000   9.757895  10.075000  16.599524  15.065000  19.480952  24.227778   

                   Jul        Aug        Sep        Oct        Nov        Dec  
station_id                                                                     
10160355000  25.487500  26.517647  24.006667  21.233333  17.164286  13.900000  
10160360000  28.999474  26.020000  23.620000  20.186957  15.754545  16.413810  
10160390000  25.577778  26.452778  23.257895  20.247619  15.171429  11.877500  
10160402000  25.028571  26.21764

In [30]:
inventory = pd.read_fwf('../data/weather/ghcnm.tavg.v3.3.0.20150624.qca.inv',header = None)

In [31]:
inventory.head()

,0,1,2,3,4,5,6,7
0,10160355000,36.93,6.95,7,SKIKDA,NaN,18U,107HIxxCO 1x-9WARM DECIDUOUS C
1,10160360000,36.83,7.82,4,ANNABA,NaN,33U,256FLxxCO 1A 7WARM CROPS C
2,10160390000,36.72,3.25,25,DAR-EL-BEIDA,NaN,34U,1365FLxxCO10A 6WARM CROPS C
3,10160395001,36.52,4.18,942,FT. NATIONAL,NaN,805R,-9MVDEno-9x-9WARM CROPS A
4,10160400001,36.80,5.10,230,CAP CARBON,NaN,28R,-9HIxxCO 1x-9WATER A


In [32]:
inventory = inventory[[0,1,2]]
inventory.columns = ['station_id','lat','lon']
inventory['lat_lon'] = zip(inventory['lat'],inventory['lon'])
inventory = inventory.drop('lat',axis=1)
inventory = inventory.drop('lon',axis=1)

In [33]:
master = station_temps.merge(inventory, how='left', left_index = True, right_on='station_id')
master = master.set_index('station_id')
master.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,lat_lon
station_id,,,,,,,,,,,,,
10160355000,12.541176,12.731250,14.538889,16.620000,19.500000,22.768421,25.487500,26.517647,24.006667,21.233333,17.164286,13.900000,"(36.93, 6.95)"
10160360000,11.159091,11.350000,13.127273,15.317391,18.513636,22.347826,28.999474,26.020000,23.620000,20.186957,15.754545,16.413810,"(36.83, 7.82)"
10160390000,11.020000,11.145238,13.392857,15.483333,18.742500,22.764286,25.577778,26.452778,23.257895,20.247619,15.171429,11.877500,"(36.72, 3.25)"
10160402000,11.385000,11.205000,13.605000,15.545000,18.475000,22.240909,25.028571,26.217647,28.499333,20.109091,19.531364,12.355556,"(36.72, 5.07)"
10160403000,9.757895,10.075000,16.599524,15.065000,19.480952,24.227778,27.222222,27.410526,23.741176,19.705000,14.585714,11.019048,"(36.47, 7.47)"


In [34]:
master.shape

(4180, 13)

In [35]:
lat = geo_data['paris']['gt_lat']
lon = geo_data['paris']['gt_lon']

In [97]:
def get_temp_from_closest_station(dest_latlon):
    distance = master['lat_lon'].apply(lambda x: haversine.haversine(x,dest_latlon))
    min_dist = distance.min()
    temps = []
    if min_dist < 150:
        closest_station = distance.argmin()
        temps = master.ix[closest_station][months]
    return temps

In [98]:
def add_temperatures_to_geo_data(geo_data):
    for dest,value in geo_data.iteritems():
        dest_latlon = (value['gt_lat'],value['gt_lon'])
        temps = get_temp_from_closest_station(dest_latlon)
        geo_data[dest]['temperatures'] = temps
    return geo_data

In [99]:
geo_temp_data = add_temperatures_to_geo_data(geo_data)

In [100]:
geo_temp_data['china']

{u'gt_lat': 39.0,
 u'gt_lon': 103.0,
 'img_path': 'static/banners/china.png',
 u'page_id': 7335,
 u'page_len': 284776,
 'temperatures': [],
 'wiki_url': u'https://en.wikivoyage.org/wiki/China'}

In [101]:
distance = master['lat_lon'].apply(lambda x: haversine.haversine(x,(geo_data['new_york_city']['gt_lat'],geo_data['new_york_city']['gt_lon'])))

In [102]:
distance.min()

4887.4426059278485

In [103]:
places = set()
for key, value in geo_temp_data.iteritems():
    length = len(value['temperatures'])
    if length > 0:
        places.add(key)


In [108]:
places

{'ardrossan_(south_australia)',
 'rhineland-palatinate',
 'kumarakom',
 'luanco',
 'bioko',
 'biysk',
 'perouges',
 'tonsberg',
 'fourni',
 'malappuram_district',
 'cyprus',
 'kerala_backwaters',
 'agia_anna_(evvia)',
 'nord-trondelag',
 'county_meath',
 'marmara_(region)',
 'samoeng',
 'kemer',
 'jerez_de_la_frontera',
 'montalcino',
 'alicante_(province)',
 'bratislava',
 'artashat',
 'samoens',
 'kottayam',
 'zilina',
 'novosedly_na_morave',
 'helsingborg',
 'rajgir',
 'castile-la_mancha',
 'ouarzazate',
 'vologda',
 'ikaalinen',
 'puttalam',
 'frogmore',
 'tiree',
 'tozeur',
 'canillo',
 'gokarna',
 'sukhothai',
 'fribourg',
 'hanau',
 'soave',
 'pleumeur-bodou',
 'lulea',
 'teignmouth',
 'paphos',
 'valladolid_(spain)',
 'nandgaon',
 'moga',
 'sykia',
 'thabo_mofutsanyana',
 'lansdowne_(india)',
 'khuldabad',
 'baarle',
 'gujranwala',
 'praiano',
 'nakhchivan_(region)',
 'maradi',
 'lahnstein',
 'walk_the_london_wall',
 'abease',
 'prokletije',
 'vigo',
 'chipping_sodbury',
 'phet

In [114]:
geo_temp_data['beijing']

{u'gt_lat': 39.9139,
 u'gt_lon': 116.392,
 'img_path': 'static/banners/beijing.png',
 u'page_id': 3187,
 u'page_len': 126995,
 'temperatures': [],
 'wiki_url': u'https://en.wikivoyage.org/wiki/Beijing'}